# Keypoint detection, Feature description and matching

In this notebook, I demonstrate how to use OpenCV's feature detection, description and matching framework. Here I am trying to run the framework on a video.

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://www.opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). You don't need this line when you write yur own C++ programs. I need it to set my interactive compiler ([Cling](https://root.cern/cling/)). For your own program, use [CMake](https://www.cmake.org/).

In [1]:
#include "../../../includeLibraries.h"

## Header inclusion for C++

In [2]:
#include <iostream>
#include <vector>
#include <stdexcept>
// #include <sstream>
// #include <string>
#include <limits>
#include <opencv2/opencv.hpp>

## Add the namespaces

In [3]:
using namespace std;

In [4]:
using namespace cv;

## Open the webcam

We are going to use the [`cv::VideoCapture` class](https://docs.opencv.org/master/d8/dfe/classcv_1_1VideoCapture.html) to grab frames from the webcam.

In [5]:
// Open the default camera (see the 0 below)
VideoCapture webcam(0);

### Any error? Is the data loaded?

In [6]:
if (!webcam.isOpened())
    throw runtime_error("OpenCV found no webcam, the program will terminate");

## 1. Grab the first frame

In [7]:
Mat reference_image;
webcam >> reference_image;

Mat reference_image_blurred;
medianBlur(reference_image, reference_image_blurred, 5);

## 1. Detect

Create a feature detector, here a ORB feature detector.

In [8]:
Ptr<FeatureDetector> detector = ORB::create();

Detect keypoints in `reference_image`.

In [9]:
vector<KeyPoint> keypoints_reference;
detector->detect(reference_image_blurred, keypoints_reference);

## 2. Describe

Create a feature descriptor, here a ORB feature descriptor

In [10]:
Ptr<DescriptorExtractor> extractor = ORB::create();

Create the feature vector for the keypoints.

In [11]:
Mat descriptors_reference;
extractor->compute(reference_image_blurred, keypoints_reference, descriptors_reference);

## 3. Run the event loop

Create a function to perform the detection, description and matching

In [12]:
Mat matchKeypoints(int aSliderValue, const Mat& aNewTestImage)
{
    // Blur the image
    Mat test_image_blurred;
    medianBlur(aNewTestImage, test_image_blurred, 5);
    
    // Create a feature detector, here a ORB feature detector.
    Ptr<FeatureDetector> detector = ORB::create();
    
    // Detect keypoints in `test_image`.
    vector<KeyPoint> keypoints_test;
    detector->detect(test_image_blurred, keypoints_test);
    
    // Create a feature descriptor, here a ORB feature descriptor
    Ptr<DescriptorExtractor> extractor = ORB::create();
    
    // Create the feature vector for the keypoints.
    Mat descriptors_test;
    extractor->compute(test_image_blurred, keypoints_test, descriptors_test);
    
    // Match keypoints in `reference_image` and `test_image` by comparing their corresponding feature vectors. 
    // Here we use a brute-force algorithm and the L2-norm.
    BFMatcher matcher(NORM_L2);
    vector<DMatch> matches;
    matcher.match(descriptors_reference, descriptors_test, matches);
        
    // Find the min and max distances between keypoints
    double max_dist = -numeric_limits<double>::max();
    double min_dist = numeric_limits<double>::max();
    
    for (int i = 0; i < matches.size(); i++ )
    {
        double dist = matches[i].distance;
        if ( dist < min_dist ) min_dist = dist;
        if ( dist > max_dist ) max_dist = dist;
    }

    // Filter the matches (only keep those with a small distance).
    vector<DMatch> good_matches;
    double threshold_distance = min_dist + (max_dist - min_dist) * aSliderValue / 100.0;

    for (int i = 0; i < matches.size(); i++ )
    {
        if (matches[i].distance < threshold_distance)
        {
            good_matches.push_back(matches[i]);
        }
    }
    
    Mat img_matches;
    drawMatches(reference_image, keypoints_reference, aNewTestImage, keypoints_test, good_matches, img_matches);
    return img_matches;
}

Create a callback for the trackbar to control the threshold

In [13]:
int slider_value = 50;

In [14]:
namedWindow("matches", WINDOW_GUI_EXPANDED); // Create a window
createTrackbar("Threshold: ", "matches", &slider_value, 100);

Run the event loop

In [15]:
while (waitKey(1) != 27)
{
    // Get a new frame
    Mat test_image;
    webcam >> test_image;
    
    // Match keypoint
    Mat img_matches = matchKeypoints(slider_value, test_image);
    imshow("matches", img_matches);
}

destroyAllWindows();

![Matches](../output.gif)